In [6]:
import pandas as pd

# Train from Jupyter

In [7]:
filt = c3.Filter().eq("technique.id", "BQ3L")
models = c3.GaussianProcessRegressionPipe.fetch({"limit": 4, "include": "id", "filter": filt}).objs

model_ids = []
for model in models:
    model_ids.append(model.id)

In [8]:
kernelLen = 18
GPR_kernel = c3.SklearnGPRKernelMatern(lengthScale=[1.0]*kernelLen, nu=0.5, coefficient=1.0).build().kernel.upsert()

GPR_technique = c3.GaussianProcessRegressionTechnique(
                    randomState=42,
                    kernel = GPR_kernel
).upsert()

GPR_dataspec = c3.GPRDataSourceSpec(
).upsert()

GPR_pipe = c3.GaussianProcessRegressionPipe(
    technique=GPR_technique
)

In [9]:
GPR_pipe.trainWithListOfAODModels(model_ids)

0

In [11]:
GPR_pipe

c3.GaussianProcessRegressionPipe(
 noTrainScore=False,
 untrainableOverride=False,
 technique=c3.GaussianProcessRegressionTechnique(
             id='BQ4L',
             meta=c3.Meta(
                    created=datetime.datetime(2022, 10, 27, 21, 4, tzinfo=datetime.timezone.utc),
                    updated=datetime.datetime(2022, 10, 27, 21, 4, tzinfo=datetime.timezone.utc),
                    timestamp=datetime.datetime(2022, 10, 27, 21, 4, tzinfo=datetime.timezone.utc)),
             version=1,
             centerTarget=False,
             validation=False,
             randomSeed=42,
             splitFraction=0.2))

# Dispatch AsyncAction

In [24]:
filt = c3.Filter().eq("technique.id", "BQ3L")
models = c3.GaussianProcessRegressionPipe.fetch({"limit": 4, "include": "id", "filter": filt}).objs

model_ids = []
for model in models:
    model_ids.append(model.id)

In [25]:
kernelLen = 18
GPR_kernel = c3.SklearnGPRKernelMatern(lengthScale=[1.0]*kernelLen, nu=0.5, coefficient=1.0).build().kernel.upsert()

GPR_technique = c3.GaussianProcessRegressionTechnique(
                    randomState=42,
                    kernel = GPR_kernel
).upsert()

GPR_dataspec = c3.GPRDataSourceSpec(
).upsert()

GPR_pipe = c3.GaussianProcessRegressionPipe(
    technique=GPR_technique,
    id="trialBig"
)

In [26]:
# action spec
spec = c3.AsyncActionSpec(
    typeName="GaussianProcessRegressionPipe",
    action="trainWithListOfAODModels",
    args={
        "modelIds": model_ids,
        "this": GPR_pipe.toJson()
    }
)

In [27]:
# submit action
action = c3.AsyncAction.submit(spec)

In [28]:
action.id

'10cf2349-2340-4af9-a047-f9f40a263856'

In [30]:
action.hasCompleted()

True

### After complete, check to see if model was trained

In [31]:
trained = c3.GaussianProcessRegressionPipe.get("trialBig")

In [32]:
trained

c3.GaussianProcessRegressionPipe(
 id='trialBig',
 meta=c3.Meta(
        tenantTagId=151,
        tenant='dev',
        tag='tc02d',
        created=datetime.datetime(2022, 10, 28, 15, 8, 54, tzinfo=datetime.timezone.utc),
        createdBy='babreu@illinois.edu',
        updated=datetime.datetime(2022, 10, 28, 15, 8, 54, tzinfo=datetime.timezone.utc),
        updatedBy='babreu@illinois.edu',
        timestamp=datetime.datetime(2022, 10, 28, 15, 8, 54, tzinfo=datetime.timezone.utc),
        fetchInclude='[]',
        fetchType='GaussianProcessRegressionPipe'),
 version=1,
 typeIdent='PIPE:LF:GPREG',
 noTrainScore=False,
 persistedModelCategory='unidentified',
 untrainableOverride=False,
 technique=c3.GaussianProcessRegressionTechnique(
             id='BHEU',
             centerTarget=False,
             validation=False,
             randomSeed=42,
             splitFraction=0.2),
 trainedModel=c3.MLTrainedModelArtifact(
                model='eJzsvddTVd23NgiKkgQBySgIiiRFEEQRdYEgQUAkKE

# Real big model now

For a N x M data set, Sklearns memory complexity is O(N^2)...

- 10,000 is too big, 30.7 TiB of memory required
- 500 too big as well, 78.3 GiB
- 400 too big, 51.3 GiB
- 300 too big, but not because of the kernel matrix... it's actually an array now, lacking 14.5 GiB
- 200 is also too much; actually complexity is (NxNxM)
- 100 still not good requiring additional 60 GiB
- 50 models finally trained!

In [54]:
filt = c3.Filter().eq("technique.id", "BQ3L")
n_models = c3.GaussianProcessRegressionPipe.fetchCount({"filter": filt})
print(n_models)

322560


In [55]:
filt = c3.Filter().eq("technique.id", "BQ3L")
models = c3.GaussianProcessRegressionPipe.fetch({"limit": 50, "include": "id", "filter": filt}).objs

model_ids = []
for model in models:
    model_ids.append(model.id)
    
kernelLen = 18
GPR_kernel = c3.SklearnGPRKernelMatern(lengthScale=[1.0]*kernelLen, nu=0.5, coefficient=1.0).build().kernel.upsert()

GPR_technique = c3.GaussianProcessRegressionTechnique(
                    randomState=42,
                    kernel = GPR_kernel
).upsert()

GPR_dataspec = c3.GPRDataSourceSpec(
).upsert()

GPR_pipe = c3.GaussianProcessRegressionPipe(
    technique=GPR_technique,
    id="trialBig50"
)

In [56]:
# action spec
spec = c3.AsyncActionSpec(
    typeName="GaussianProcessRegressionPipe",
    action="trainWithListOfAODModels",
    args={
        "modelIds": model_ids,
        "this": GPR_pipe.toJson()
    }
)

# submit action
action = c3.AsyncAction.submit(spec)

print(action.id)

2e9bff7f-87ab-46d9-8087-1cef0111bd4b


# Exclude features

In [1]:
filt = c3.Filter().eq("technique.id", "BQ3L")
models = c3.GaussianProcessRegressionPipe.fetch({"limit": 4, "include": "id", "filter": filt}).objs

model_ids = []
for model in models:
    model_ids.append(model.id)
    
kernelLen = 18
GPR_kernel = c3.SklearnGPRKernelMatern(lengthScale=[1.0]*kernelLen, nu=0.5, coefficient=1.0).build().kernel.upsert()

GPR_technique = c3.GaussianProcessRegressionTechnique(
                    randomState=42,
                    kernel = GPR_kernel
).upsert()

GPR_dataspec = c3.GPRDataSourceSpec(
).upsert()

GPR_pipe = c3.GaussianProcessRegressionPipe(
    technique=GPR_technique,
    id="Big4"
)

In [2]:
excludeFeatures = ["acure_ait_width", "acure_cloud_ph"]

In [3]:
# action spec
spec = c3.AsyncActionSpec(
    typeName="GaussianProcessRegressionPipe",
    action="trainWithListOfAODModels",
    args={
        "modelIds": model_ids,
        "this": GPR_pipe.toJson(),
        "excludeFeatures": excludeFeatures
    }
)

# submit action
action = c3.AsyncAction.submit(spec)

print(action.id)

a5dfffad-0569-44cc-b6e4-1029908a6b4b
